# Multi-class classifiaction of RGB images of "Rock, Paper, Scissors" hand signs using Flux.jl and Deep Neural Networks

## Initialization

### Install and load packages

In [2]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

using Plots, Interact, Flux
using Flux: onehotbatch, argmax, crossentropy, throttle, mse, onecold, shuffle
using Flux.Data: DataLoader
using IterTools: ncycle, partition
using JLD2, FileIO, ImageIO

theme(
    :wong;
    label="",
    markerstrokewidth=0.3,
    markerstrokecolor=:white
)

### Load Data

In [9]:
X_rock = load("data/rock.jld2")["rock_imgs"]
X_paper = load("data/paper.jld2")["paper_imgs"]
X_scissors = load("data/scissors.jld2")["scissors_imgs"]

println("Data loaded")

## Functions

In [29]:
function e_i(i::Integer, n::Integer, T::Type=Int64)
    v = zeros(T, n)
    v[i] = 1

    return v
end
function array2matrix(X::AbstractArray{T,4}) where T
    height, width, rgb, n_samples = size(X)

    Y = zeros(height * width * rgb, n_samples)

    for idx in 1:n_samples
        Y[:, idx] = vec(X[:, :, :, idx])
    end

    return Y
end

println("Functions defined")

## Reshape Data

In [27]:
X = [X_rock, X_paper, X_scissors]
labels = ["rock", "paper", "scissors"]

label2vector = Dict{String,Vector{Float64}}()
for (i, l) in enumerate(labels)
    label2vector[l] = e_i(i, length(labels))
end
vector2label = Dict((v, k) for (k, v) in label2vector)

train_fraction = 0.6
n_samples = size(X_rock, 4)
n_train = Int(n_samples * train_fraction)
n_test = n_samples - n_train

X_train = hcat((array2matrix(Xi[:, :, :, 1:n_train]) for Xi in X)...)
X_test = hcat((array2matrix(Xi[:, :, :, (n_train + 1):n_samples]) for Xi in X)...)

train_labels = hcat((repeat(label2vector[l], 1, n_train) for l in labels)...)
test_labels  = hcat((repeat(label2vector[l], 1, n_test ) for l in labels)...)

@show train_labels[:,1]
Y_train = onehotbatch(train_labels, 0:2)

println("Training and test data separated")

## Train

In [ ]:
m = Chain(
    Dense(28^2, 40, relu),
    Dense(40, 10),
    softmax
)

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

opt = ADAM()

function evalcb()
    l = round(loss(X_train, Y_train); digits=4)
    a = round(accuracy(X_train, Y_train); digits=4)
    println("Loss: $(l)\tAccuracy: $(a)")
end

epochs = 8 # each epoch represents one pass through data
batchsize = 100

train_loader = DataLoader(X_train, Y_train; batchsize=batchsize)
Flux.train!(loss, params(m), ncycle(train_loader, epochs), opt; cb=throttle(evalcb, 2.0))

## Review